# Pelatihan Model Klasifikasi Gambar Bahasa Isyarat

## Pengumpulan Dataset

Dataset untuk membuat model yang akan mengklasifikasikan gambar tangan (dari pergelangan tangan hingga per ujung jari) menjadi huruf alfabet dalam bahasa isyarat adalah [Sign Language MNIST](https://www.kaggle.com/datasets/datamunge/sign-language-mnist) diambil dari penyedia dataset MNIST lewat laman [Kaggle](https://www.kaggle.com). Dataset tersebut berisikan data latih yang berjumlah 27.455 dan data tes berjumlah 7172 yang keduanya memiliki label berupa angka dari 0-25 yang tiap angkanya secara berurutan mewakilkan huruf huruf alfabet dari A-Z kecuali huruf J dengan label 9 dan huruf z dengan label 25 dikarenakan huruf tersebut dalam bahasa isyarat menggunakan gestur, bukan gambar. Dataset [Sign Language MNIST](https://www.kaggle.com/datasets/datamunge/sign-language-mnist) berisikan gambar yang sudah diubah menjadi barisan bilangan bulat dalam format grayscale (tiap nilai pada larik/array yaitu 0-255) dengan ukuran 28x28 pixel dan format file berupa .csv dengan kolom berjumlah 785 dimulai dari "label, pixel1, pixel2, pixel3...pixel784".

Unduh dataset [Sign Language MNIST](https://www.kaggle.com/datasets/datamunge/sign-language-mnist) dari laman [Kaggle](https://www.kaggle.com) dan ekstrak file terkompresinya.

## Muat Pakcage

Setelah mengunduh dataset, import semua package yang dibutuhkan untuk memproses data dan membuat model.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pandas as pd

## Muat Dataset

Setelah memuat semua package yang dibutuhkan, maka buat variabel untuk memuat dataset.

In [ ]:
train_df = pd.read_csv("sign_mnist_train/sign_mnist_train.csv")
test_df = pd.read_csv("sign_mnist_test/sign_mnist_test.csv")

Dikarenakan dataset sudah terpisah antara data latih dan data tes, maka untuk menyiapkan variabel tersebut tidak diperlukan langkah tambahan dalam memisahkan data.
Kemudian pisahkan antara label tidak data dengan data latih

In [ ]:
y_train = train_df['label']
y_test = test_df['label']
del train_df['label']
del test_df['label']

Setelah memisahkan label dengan data inputan, label dengan bilangan positif yang mewakili tiap huruf (0-25 : A-Z) akan diubah menjadi larik/array.
misal huruf "A" yang dilabeli dengan angka "0" akan diubah menjadi larik/array 

"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"

 dan huruf "B" menjadi larik/array
 
 "[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]" dan seterusnya sampai huruf terakhir.
 
 Proses ini disebut binerisasi label yang menghasilkan label dalam bentuk larik/array angka biner yang disebut "One-Hot".

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

Beralih ke data latih dan data tes yang sudah dipisahkan dengan label, data tersebut berisi bilangan positif mulai dari 0-255 untuk tiap pixel gambar. Angka tersebut akan diubah dari rentang angka 0-255 menjadi 0-1 supaya mempermudah perhitungan dalam model. Proses ini dinamakan "Normalization" atau Normalisasi data inputan.

In [ ]:
x_train = train_df.values
x_test = test_df.values

x_train = x_train / 255
x_test = x_test / 255

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28,28,1)

Dalam melatih model, terutama yang akan digunakan adalah [pembelajaran mendalam / deep learning](https://www.dicoding.com/blog/mengenal-deep-learning/), prinsip yang bekerja adalah semakin banyak jumlah dan variasi data yang dilatih, maka semakin baik pula hasil prediksi dari sebuah model. Oleh karena itu, dataset saat ini akan diperbanyak/[augmentasi](https://blog.algorit.ma/augmentasi-data/) dengan rincian sebagai berikut.
1. Secara random menambah variasi rotasi gambar/data dengan perubahan 10 derajat. [rotation_range=10]
2. Secara random menambah variasi zoom gambar/data dengan perubahan 10%. [zoom_range=0.1]
3. Secara random menambah variasi pergeseran gambar dari titik tengah secara horisontal sebesar 10% dari lebar gambar. [width_shift_range=0.1]
4. Secara random menambah variasi pergeseran gambar dari titik tengah secara vertikal sebesar 10% dari tinggi gambar. [height_shift_range=0.1]

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    # randomly rotate images in the range (degrees, 0 to 180)
    rotation_range=10,
    zoom_range=0.1,  # Randomly zoom image
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

## Perancangan Model

Model yang akan digunakan adalah model deep learning dengan arsitektur [Convolutional Neural Network](https://medium.com/@16611110/apa-itu-convolutional-neural-network-836f70b193a4) atau bisa disingkat [CNN](https://medium.com/@16611110/apa-itu-convolutional-neural-network-836f70b193a4).

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.00001)

model = keras.models.Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding='same',
          activation='relu', input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding='same'))
model.add(Conv2D(50, (3, 3), strides=1, padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding='same'))
model.add(Conv2D(25, (3, 3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding='same'))
model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=24, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

## Pelatihan Model

Kemudian melatih model dengan code dibawha

In [ ]:
history = model.fit(datagen.flow(x_train, y_train, batch_size = 128) ,epochs = 20 , validation_data = (x_test, y_test) , callbacks = [learning_rate_reduction])

Selanjutnya menyimpan model yang sudah dilatih ke dalam sebuah file untuk digunakan di dalam aplikasi. Model disimpan dengan format keras model dengan ekstensi file .h5